# Voice Cloning

I'll be attempting to make a voice cloning model using Coqui TTS and the TIMIT dataset. Documentation on how to use Coqui TTS can be found here: https://tts.readthedocs.io/en/latest/index.html#.

In [1]:
import os

from trainer import Trainer, TrainerArgs

from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.vits import Vits, VitsArgs, VitsAudioConfig
from TTS.tts.utils.speakers import SpeakerManager
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

C:\Users\Chris\Anaconda3\envs\torch\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')


# Data Preparation

In [2]:
dataset_path = 'D:\data\TIMIT\TRAIN'
output_path = './output'

I'll be using the vctk dataset config since it is already formatted for a multispeaker dataset. If you're planning on using this dataset config, your dataset needs to be configured in this way:

/MyTTSDataset\
&emsp;| -> /txt\
&emsp;&emsp;&emsp;| -> /speaker\
&emsp;&emsp;&emsp;&emsp;&emsp;| -> audio1.txt\
&emsp;&emsp;&emsp;&emsp;&emsp;| -> audio2.txt\
&emsp;&emsp;&emsp;&emsp;&emsp;| -> ...\
&emsp;| -> /wav48\
&emsp;&emsp;&emsp;| -> /speaker\
&emsp;&emsp;&emsp;&emsp;&emsp;| -> audio1.wav\
&emsp;&emsp;&emsp;&emsp;&emsp;| -> audio2.wav\
&emsp;&emsp;&emsp;&emsp;&emsp;| -> ...

In [3]:
# define dataset config
# need to use vctk_old format if using wav files instead of flac

dataset_config = BaseDatasetConfig(
    name="vctk_old", meta_file_train="", language="en-us", path=dataset_path
)

In [4]:
audio_config = VitsAudioConfig(
    sample_rate=22050, win_length=1024, hop_length=256, num_mels=80, mel_fmin=0, mel_fmax=None
)

In [5]:
vitsArgs = VitsArgs(
    use_speaker_embedding=True,
)

In [6]:
config = VitsConfig(
    model_args=vitsArgs,
    audio=audio_config,
    run_name="vits_vctk",
    batch_size=32,
    eval_batch_size=16,
    batch_group_size=5,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=1000,
    text_cleaner="english_cleaners",
    use_phonemes=True,
    phoneme_language="en",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    compute_input_seq_cache=True,
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    max_text_len=325,  # change this if you have a larger VRAM than 16GB
    output_path=output_path,
    datasets=[dataset_config],
    cudnn_benchmark=False,
)

In [7]:
# INITIALIZE THE AUDIO PROCESSOR
# Audio processor is used for feature extraction and audio I/O.
# It mainly serves to the dataloader and the training loggers.
ap = AudioProcessor.init_from_config(config)

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [8]:
# INITIALIZE THE TOKENIZER
# Tokenizer is used to convert text to sequences of token IDs.
# config is updated with the default characters if not defined in the config.
tokenizer, config = TTSTokenizer.init_from_config(config)

In [9]:
# LOAD DATA SAMPLES
# Each sample is a list of ```[text, audio_file_path, speaker_name]```

train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

 | > Found 2580 files in D:\data\TIMIT\TRAIN


In [10]:
# init speaker manager for multi-speaker training
# it maps speaker-id to speaker-name in the model and data-loader
speaker_manager = SpeakerManager()
speaker_manager.set_ids_from_data(train_samples + eval_samples, parse_key="speaker_name")
config.model_args.num_speakers = speaker_manager.num_speakers

In [11]:
# init model
model = Vits(config, ap, tokenizer, speaker_manager)

 > initialization of speaker-embedding layers.


In [ ]:
# init the trainer and 🚀
trainer = Trainer(
    TrainerArgs(),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)
trainer.fit()

 > Using CUDA: True
 > Number of GPUs: 1

 > Model has 86476460 parameters

 > EPOCH: 0/1000
 --> ./output\vits_vctk-September-23-2022_02+46AM-3c624ce


 > `speakers.pth` is saved to ./output\vits_vctk-September-23-2022_02+46AM-3c624ce\speakers.pth.
 > `speakers_file` is updated in the config.json.
[*] Pre-computing phonemes...


  0%|          | 1/2555 [00:00<24:54,  1.71it/s]

['<BLNK>', 'b', '<BLNK>', 'ɪ', '<BLNK>', 'f', '<BLNK>', 'ɔ', '<BLNK>', 'ɹ', '<BLNK>', ' ', '<BLNK>', 'm', '<BLNK>', 'ʌ', '<BLNK>', 't', '<BLNK>', '͡', '<BLNK>', 'ʃ', '<BLNK>', ' ', '<BLNK>', 'l', '<BLNK>', 'ɔ', '<BLNK>', 'ŋ', '<BLNK>', 'ɡ', '<BLNK>', 'ɚ', '<BLNK>', ' ', '<BLNK>', 'ð', '<BLNK>', 'ə', '<BLNK>', ' ', '<BLNK>', 'm', '<BLNK>', 'ɚ', '<BLNK>', 'i', '<BLNK>', 'n', '<BLNK>', ' ', '<BLNK>', 'k', '<BLNK>', 'w', '<BLNK>', 'a', '<BLNK>', 'ɪ', '<BLNK>', 'ə', '<BLNK>', 't', '<BLNK>', 'ɪ', '<BLNK>', 'd', '<BLNK>', ' ', '<BLNK>', 'd', '<BLNK>', 'a', '<BLNK>', 'ʊ', '<BLNK>', 'n', '<BLNK>', '.', '<BLNK>']
 [!] Character '͡' not found in the vocabulary. Discarding it.


100%|██████████| 2555/2555 [01:35<00:00, 26.83it/s]


...


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.00901 (-0.00000)
     | > avg_loss_disc: 2.66726 (+0.01700)
     | > avg_loss_disc_real_0: 0.20332 (+0.09268)
     | > avg_loss_disc_real_1: 0.22393 (+0.01589)
     | > avg_loss_disc_real_2: 0.27154 (+0.01429)
     | > avg_loss_disc_real_3: 0.24568 (+0.04543)
     | > avg_loss_disc_real_4: 0.26410 (+0.01336)
     | > avg_loss_disc_real_5: 0.26303 (+0.01284)
     | > avg_loss_0: 2.66726 (+0.01700)
     | > avg_loss_gen: 2.00895 (+0.19610)
     | > avg_loss_kl: 1.65556 (+0.07859)
     | > avg_loss_feat: 2.61618 (-0.41200)
     | > avg_loss_mel: 17.88013 (-0.93196)
     | > avg_loss_duration: 2.00462 (-0.02112)
     | > avg_loss_1: 26.16546 (-1.09038)


 > EPOCH: 235/1000
 --> ./output\vits_vctk-September-23-2022_02+46AM-3c624ce




> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en
		| > phoneme backend: gruut
	| > 1 not found characters:
	| > ͡
| > Number of instances : 2555



 > TRAINING (2022-09-24 01:13:47) 


 | > Preprocessing samples
 | > Max text length: 81
 | > Min text length: 16
 | > Avg text length: 49.37025440313111
 | 
 | > Max audio length: 171808.0
 | > Min audio length: 21798.0
 | > Avg audio length: 67219.45401174168
 | > Num. instances discarded samples: 0
 | > Batch group size: 160.



   --> STEP: 0/80 -- GLOBAL_STEP: 18800
     | > loss_disc: 2.70787  (2.70787)
     | > loss_disc_real_0: 0.18303  (0.18303)
     | > loss_disc_real_1: 0.20511  (0.20511)
     | > loss_disc_real_2: 0.25571  (0.25571)
     | > loss_disc_real_3: 0.23978  (0.23978)
     | > loss_disc_real_4: 0.23067  (0.23067)
     | > loss_disc_real_5: 0.23985  (0.23985)
     | > loss_0: 2.70787  (2.70787)
     | > grad_norm_0: 159.85724  (159.85724)
     | > loss_gen: 1.83592  (1.83592)
     | > loss_kl: 1.19972  (1.19972)
     | > loss_feat: 3.00049  (3.00049)
     | > loss_mel: 18.62750  (18.62750)
     | > loss_duration: 1.45286  (1.45286)
     | > amp_scaler: 128.00000  (128.00000)
     | > loss_1: 26.11650  (26.11650)
     | > grad_norm_1: 910.97003  (910.97003)
     | > current_lr_0: 0.00019 
     | > current_lr_1: 0.00019 
     | > step_time: 3.54620  (3.54623)
     | > loader_time: 23.70830  (23.70827)


   --> STEP: 25/80 -- GLOBAL_STEP: 18825
     | > loss_disc: 2.64383  (2.70995)
     | > lo

In [13]:
%load_ext tensorboard

In [14]:
%tensorboard --logdir="./output\run-September-15-2022_04+45AM-910d77a"